### 개발 환경설정

In [32]:
!pip install --upgrade pip
!pip install konlpy
!pip install transformers
!pip install -U sentence-transformers
!pip install korean-name-generator
!pip install tokenizers

In [2]:
import pandas as pd
import re
import numpy as np
import pandas as pd
import json
import time
import torch
from konlpy.tag import Kkma
from konlpy.utils import pprint
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans

### 데이터 불러오기 및 가공

In [3]:
path="/content/hana_data6.csv"
df = pd.read_csv(path)

In [4]:
df['text'] = df['cont_title'] + ' ' + df['cont_explanation']
df=df.dropna(subset=['text'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2953 entries, 0 to 2998
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        2953 non-null   int64 
 1   cont_id           2953 non-null   int64 
 2   cont_title        2953 non-null   object
 3   cont_url          2953 non-null   object
 4   cont_author       2953 non-null   object
 5   cont_date         2953 non-null   object
 6   cont_explanation  2953 non-null   object
 7   text              2953 non-null   object
dtypes: int64(2), object(6)
memory usage: 207.6+ KB


In [44]:
# 10개의 빈 DataFrame 생성
dfs = [pd.DataFrame() for _ in range(10)]

# df의 각 행을 10개의 DataFrame에 분배
for i, row in df.iterrows():
     dfs[i % 10] = pd.concat([dfs[i % 10], pd.DataFrame([row])], ignore_index=True) # Use pd.concat to append rows

# 각 DataFrame을 df1부터 df10까지 변수에 저장
for i in range(10):
    globals()[f'df{i+1}'] = dfs[i]

In [45]:
example = df1[['cont_date', 'text']].to_dict('records')

In [52]:
 df1

,Unnamed: 0,cont_id,cont_title,cont_url,cont_author,cont_date,cont_explanation,text
0,0,4414795,에코프로비엠 44조 양극재 계약 - 개관들 꼴좋게 됐네,https://cafe.naver.com/ustock/4414795,맨유공장,2023.12.02. 00:14,개미들의 2차전지 죽일려고 쌩난리를 치더니...\n그래서 되지도 않는 반도체를 작년...,에코프로비엠 44조 양극재 계약 - 개관들 꼴좋게 됐네 개미들의 2차전지 죽일려고 ...
1,10,4414751,12/1 시외 등락율 순위 최종마감,https://cafe.naver.com/ustock/4414751,거먼천사,2023.12.01. 19:22,시장이 비정상적 ......우선주의 날이군요...ㅋㅋ,12/1 시외 등락율 순위 최종마감 시장이 비정상적 ......우선주의 날이군요...ㅋㅋ
2,20,4414703,"결국 올 것이 왔다…""LG, 돈 더 내놔라"" GM의 초강수",https://cafe.naver.com/ustock/4414703,페레즈쿠르즈,2023.12.01. 16:40,"LG엔솔 받을 보조금\nGM ""절반 넘게 달라""\n'과잉 청구서' 복병 만난 K배터...","결국 올 것이 왔다…""LG, 돈 더 내놔라"" GM의 초강수 LG엔솔 받을 보조금\n..."
3,30,4414589,흘러 내리네요,https://cafe.naver.com/ustock/4414589,달빛이라도,2023.12.01. 13:25,주가가 올랐으니\n내려야죠\n고스닥820\n최소 780 부터 추매 하려고요\n오늘은...,흘러 내리네요 주가가 올랐으니\n내려야죠\n고스닥820\n최소 780 부터 추매 하...
4,40,4414488,지금 ls머티리얼 청약 잘 되시나요?,https://cafe.naver.com/ustock/4414488,겨울잘지내기,2023.12.01. 10:41,\n* 종목추천글은 본인등급의 회원 종목방에 글을 올려야 합니다.\n* 누구에게든 ...,지금 ls머티리얼 청약 잘 되시나요? \n* 종목추천글은 본인등급의 회원 종목방에 ...
...,...,...,...,...,...,...,...,...
291,2950,4385128,"AI 및 데이터로 보는 기업종목, 9월18일(금) 대주주 변화량 외",https://cafe.naver.com/ustock/4385128,스티븐,2023.09.18. 09:20,--\n* 주식추천글은 본인등급의 회원추천방에 글을 올려야 합니다.\n* 누구에게든...,"AI 및 데이터로 보는 기업종목, 9월18일(금) 대주주 변화량 외 --\n* 주식..."
292,2960,4385007,"외국인, 기관, 개인이 순매수/순매도 상위종목이네요!!!!!(9.15기준)",https://cafe.naver.com/ustock/4385007,안정된삶,2023.09.18. 07:04,--\n* 주식추천글은 본인등급의 회원추천방에 글을 올려 주셔야 합니다.\n* 누구...,"외국인, 기관, 개인이 순매수/순매도 상위종목이네요!!!!!(9.15기준) --\n..."
293,2970,4384927,치고빠지고..낄끼빠빠 잘하는게..,https://cafe.naver.com/ustock/4384927,코코온냐,2023.09.17. 17:00,--\n*. 종목 추천과 종목 A/S를 제외한 순수 주식 이야기방입니다.\n* 종목...,치고빠지고..낄끼빠빠 잘하는게.. --\n*. 종목 추천과 종목 A/S를 제외한 순...
294,2980,4384834,이수페타시스 내년 11월까지 보면 어떤가요?,https://cafe.naver.com/ustock/4384834,러키몰리,2023.09.17. 00:42,--\n*. 종목 추천과 종목 A/S를 제외한 순수 주식 이야기방입니다.\n* 종목...,이수페타시스 내년 11월까지 보면 어떤가요? --\n*. 종목 추천과 종목 A/S를...


### TF-IDF 키워드 추출 클래스 정의

In [46]:
class TFIDF:
  def __init__(self,list_dict):
    self.list_dict=list_dict
  def default_vector_tfidf(self,tokenizer=None):
    if tokenizer:
      corpus=[re.sub("[\n,\t]"," ",d['text']) for d in self.list_dict]
      vector = CountVectorizer(tokenizer=tokenizer)
    else:
      corpus=[re.sub("[\n,\t]"," ",d['text']) for d in self.list_dict]
      vector = CountVectorizer()

    keyword_list=[]
    for i,row in  enumerate(vector.transform(corpus).toarray()):
      vector.vocabulary_
      keywords=[ list(vector.vocabulary_.keys())[list(vector.vocabulary_.values()).index(val)]  for val in  np.argsort(row)[-15:]]
      keyword_list.append({'cont_date':self.list_dict['cont_date'],'text':self.list_dict['text'],'keywords':keywords })
      print(keywords)
    return keyword_list
  def  tfidf_vector_tfidf(self,tokenizer=None):
    corpus=[re.sub("[\n,\t]"," ",d['text']) for d in self.list_dict]
    tokenized_corpus = [tokenizer(doc) for doc in corpus]
    preprocessed_corpus = [' '.join(tokens) for tokens in tokenized_corpus]
    vector = TfidfVectorizer().fit(preprocessed_corpus)

    keyword_list=[]
    for i,row in  enumerate(vector.transform(corpus).toarray()):
      vector.vocabulary_
      keywords=[ list(vector.vocabulary_.keys())[list(vector.vocabulary_.values()).index(val)]  for val in  np.argsort(row)[-15:]]
      keyword_list.append({'cont_date':self.list_dict[i]['cont_date'],'text':self.list_dict[i]['text'],'keywords':keywords })
      print(keywords)
    return keyword_list
  def run(self,vectorizer_type="default",tokenizer=None):
    if vectorizer_type=="default":
      self.keyword_list=self.default_vector_tfidf(tokenizer=tokenizer)
    elif vectorizer_type=="tfidf":
      self.keyword_list=self.tfidf_vector_tfidf(tokenizer=tokenizer)
    return self.keyword_list

### 자연어 처리 및 토큰화 실험

In [49]:
# 불용어 파일 불러오기
file_path = '/content/stopwords.txt'

stopwords_txt = []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        stopwords_txt.append(line.strip())

In [58]:
import re
from konlpy.tag import Kkma

# stopwords.txt 파일을 읽어와서 stopwords 리스트를 만듭니다.
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords_txt = f.read().splitlines()

def kor_exclude_stopwords(word):
    stopwords = stopwords_txt
    if word in stopwords:
        return False
    else:
        return True

def tokenizer(sent):
    kor_regrex = r"[ㄱ-ㅎ,가-힣]+"
    sent = re.sub('[,\[\]\(\)\{\}]', ' ', sent)
    kor = re.findall(kor_regrex, sent)

    kkma = Kkma()
    kor = kkma.nouns(' '.join(kor))
    kor_filtered = []
    for k in kor:
        if kor_exclude_stopwords(k):
            kor_filtered.append(k)
        else:
            print(f"불용어로 필터링된 단어: {k}")  # 디버깅용 출력

    return kor_filtered

# 테스트 문장
sent = "이것은 테스트 문장입니다. 불용어가 제대로 필터링 되는지 확인해보세요."
tokens = tokenizer(sent)
print(tokens)


불용어로 필터링된 단어: 이것
['테스트', '문장', '불', '불용어', '용어', '제대', '필터링', '확인']


In [73]:
def kor_exclude_stopwords(word):
  stopwords = stopwords_txt
  if word in stopwords:
        return False
  else:
        return True
def tokenizer(sent):
  kor_regrex=r"[ㄱ-ㅎ,가-힣]+"
  sent=re.sub('[,\[\]\(\)\{\}]',' ',sent)
  kor=re.findall(kor_regrex,sent)
  #kor는  morph.noun 쓰기
  kkmm=Kkma()
  kor_filtered = []
  for k in kor:
        if kor_exclude_stopwords(k):
            kor_filtered.append(k)
        else: # Align the else statement with the if statement
            pass # Add a placeholder, or remove the else block if not needed
  return kor_filtered


In [74]:
# 실험
tfidf=TFIDF(example)
keyword_list = []
keyword_list=tfidf.run(vectorizer_type="tfidf",tokenizer=tokenizer)

['만들고', '밀더니', '종목들만', '담주', '죽일려고', '기형적인', '하겠네', '쌩난리를', '반도체를', '조단위인', '한미반도체같이', '치더니', '영업익에', '볼만', '개관들']
['방한', '방산주', '방송에서', '힘이', '방안을', '방안이', '시외', '등락율', '시장이', '순위', '최종마감', '우선주의', '비정상적', 'ㅋㅋ', '날이군요']
['합작공장', '합작공장의', '세액공제', '공유', '최대', '생산', '북미', '알려졌다', '미국', '배당', '관계자는', '합작법인', '것으로', '완성차', '배터리']
['방송에서', '방산', '하세요', '오늘은', '주가가', '최소', '추매', '성투', '내리네요', '끝일듯요', '하려고요', '영업', '흘러', '올랐으니', '내려야죠']
['회원', '종목방에', '종목추천글은', '제가', '청약', '되시나요', '조회가', '넣고', '문자는', '왔는데', '청약신청했다고', '청약신청내역에서', '못하는건지', '눌렀는데', '안되어서요']
['방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방문해도', '방문', '힘이', '셀트리온']
['방해하거나', '방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방문한', '힘이', '시가총액']
['생김', '종목변경', '저렴한', '윈도드레싱', '사이에', '왕왕있음', '동시호가가', '지켜보다가', '현물사이', '직행', '익절완료', '괴리율이', '동시호가에', '주식선물과', '주식선물은']
['방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방향', '힘이', '신규계약

In [75]:
def kor_exclude_stopwords(word):
  stopwords= stopwords_txt + ["분",",","종목추천글은 본인등급의 회원 종목방에 글을 올려야 합니다.",
              "누구에게든 비방글, 저격글, 욕설글, 비꼬는글, 분란조장글, 홍보글, 금전글 등은 통보없이 글 삭제와 강퇴 됩니다.",
              "카페내의 글들은 참고만 하셔야 하며 투자의 책임은 본인에게 있습니다.","ㅋ","ㅎ", "여파", "열람", "정답","[0-9]","^[가-힣]+대박[가-힣]+$","의","을", "줘라", "추천주", "소통방", "대박","대박투자"]
  if word in stopwords:
        return False
  else:
        return True
def tokenizer(sent):
  kor_regrex=r"[ㄱ-ㅎ,가-힣]+"
  sent=re.sub('[,\[\]\(\)\{\}]',' ',sent)
  kor=re.findall(kor_regrex,sent)
  #kor는  morph.noun 쓰기
  kkmm=Kkma()
  kor_filtered = []
  for k in kor:
        if kor_exclude_stopwords(k):
            kor_filtered.append(k)
        else: # Align the else statement with the if statement
            pass # Add a placeholder, or remove the else block if not needed
  return kor_filtered


In [76]:
# 실험
tfidf=TFIDF(example)
keyword_list = []
keyword_list=tfidf.run(vectorizer_type="tfidf",tokenizer=tokenizer)

['죽일려고', '하겠네', '쌩난리를', '한미반도체같이', '조단위인', '반도체를', '치더니', '영업익에', '만들고', '담주', '작년부터', '볼만', '쥐꼬리만한', '밀더니', '기형적인']
['방산', '방역당국', '방산업체향', '방산주', '방송에서', '방안을', '시외', '등락율', '순위', '시장이', '최종마감', '우선주의', '비정상적', 'ㅋㅋ', '날이군요']
['합작공장', '말했다', '세액공제', '공유', '최대', '생산', '북미', '알려졌다', '미국', '배당', '합작법인', '관계자는', '것으로', '완성차', '배터리']
['방법으로', '방문해도', '하세요', '오늘은', '주가가', '최소', '추매', '성투', '흘러', '영업', '올랐으니', '하려고요', '끝일듯요', '내리네요', '내려야죠']
['회원', '종목방에', '종목추천글은', '제가', '청약', '청약신청했다고', '안되어서요', '넣고', '못하는건지', '눌렀는데', '되시나요', '왔는데', '문자는', '조회가', '청약신청내역에서']
['방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방문해도', '방문', '힘이', '셀트리온']
['방향', '방해하거나', '방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방문한', '시가총액']
['미친듯이', '윈도드레싱', '저렴한', '사이에', '생김', '아무래도', '왕왕있음', '현물사이', '지켜보다가', '가격에', '익절완료', '괴리율이', '주식선물과', '동시호가에', '주식선물은']
['방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방향', '힘이',

In [77]:
def kor_exclude_stopwords(word):
  stopwords= stopwords_txt + ["분",",","종목추천글은 본인등급의 회원 종목방에 글을 올려야 합니다.",
              "누구에게든 비방글, 저격글, 욕설글, 비꼬는글, 분란조장글, 홍보글, 금전글 등은 통보없이 글 삭제와 강퇴 됩니다.",
              "카페내의 글들은 참고만 하셔야 하며 투자의 책임은 본인에게 있습니다.","ㅋ","ㅎ", "여파", "열람", "정답","[0-9]","^[가-힣]+대박[가-힣]+$","의","을", "줘라", "추천주", "소통방", "대박","대박투자"]
  if any(sw in word for sw in stopwords):
        return False
  else:
        return True
def tokenizer(sent):
  kor_regrex=r"[ㄱ-ㅎ,가-힣]+"
  sent=re.sub('[,\[\]\(\)\{\}]',' ',sent)
  kor=re.findall(kor_regrex,sent)
  kor_string = ' '.join(kor)
  kkmm=Kkma()
  pos = kkmm.pos(kor_string)
  kor = []
  for keyword, type in pos:
    # 고유명사만 남기기
    if type == "NNP":
      kor. append (keyword)
  kor_filtered = []
  for k in kor:
        if kor_exclude_stopwords(k):
            kor_filtered.append(k)
        else: # Align the else statement with the if statement
            pass # Add a placeholder, or remove the else block if not needed
  return kor_filtered

In [81]:
import re
from konlpy.tag import Kkma

# stopwords.txt 파일을 읽어와서 stopwords 리스트를 만듭니다.
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords_txt = f.read().splitlines()

def kor_exclude_stopwords(word):
    stopwords = stopwords_txt
    if any(sw in word for sw in stopwords):
        return False
    else:
        return True

def tokenizer(sent):
    kor_regrex = r"[ㄱ-ㅎ가-힣]+"
    sent = re.sub('[,\[\]\(\)\{\}]', ' ', sent)
    kor = re.findall(kor_regrex, sent)
    kor_string = ' '.join(kor)

    kkma = Kkma()
    pos = kkma.pos(kor_string)
    kor = []

    for keyword, type in pos:
        # 고유명사만 남기기
        if type == "NNG":
            kor.append(keyword)
    print(kor)
    kor_filtered = []
    for k in kor:
        if kor_exclude_stopwords(k):
            kor_filtered.append(k)
        else:
            print(f"불용어로 필터링된 단어: {k}")  # 디버깅용 출력

    if not kor_filtered:
        print("모든 단어가 불용어로 필터링되었습니다.")

    return kor_filtered

# 테스트 문장
sent = "이것은 테스트 문장입니다. 불용어가 제대로 필터링 되는지 확인해보세요 백신.에이치앤비디자인"
tokens = tokenizer(sent)
print("최종 토큰 리스트:", tokens)


['테스트', '문장', '불', '용어', '제대', '필터링', '확인', '보세', '백신', '이치', '앤', '비', '디자인']
불용어로 필터링된 단어: 테스트
불용어로 필터링된 단어: 문장
불용어로 필터링된 단어: 불
불용어로 필터링된 단어: 용어
불용어로 필터링된 단어: 제대
불용어로 필터링된 단어: 필터링
불용어로 필터링된 단어: 확인
불용어로 필터링된 단어: 보세
불용어로 필터링된 단어: 백신
불용어로 필터링된 단어: 이치
불용어로 필터링된 단어: 앤
불용어로 필터링된 단어: 비
불용어로 필터링된 단어: 디자인
모든 단어가 불용어로 필터링되었습니다.
최종 토큰 리스트: []


In [78]:
# 실험
tfidf=TFIDF(example)
keyword_list = []
keyword_list=tfidf.run(vectorizer_type="tfidf",tokenizer=tokenizer)

ValueError: empty vocabulary; perhaps the documents only contain stop words

## 토픽 추출 및 병합

In [82]:
def kor_exclude_stopwords(word):
  stopwords= stopwords_txt + ["분",",","종목추천글은 본인등급의 회원 종목방에 글을 올려야 합니다.",
              "누구에게든 비방글, 저격글, 욕설글, 비꼬는글, 분란조장글, 홍보글, 금전글 등은 통보없이 글 삭제와 강퇴 됩니다.",
              "카페내의 글들은 참고만 하셔야 하며 투자의 책임은 본인에게 있습니다.","ㅋ","ㅎ", "여파", "열람", "정답","[0-9]","^[가-힣]+대박[가-힣]+$","의","을", "줘라", "추천주", "소통방", "대박","대박투자"]
  if word in stopwords:
        return False
  else:
        return True
def tokenizer(sent):
  kor_regrex=r"[ㄱ-ㅎ,가-힣]+"
  sent=re.sub('[,\[\]\(\)\{\}]',' ',sent)
  kor=re.findall(kor_regrex,sent)
  #kor는  morph.noun 쓰기
  kkmm=Kkma()
  kor_filtered = []
  for k in kor:
        if kor_exclude_stopwords(k):
            kor_filtered.append(k)
        else: # Align the else statement with the if statement
            pass # Add a placeholder, or remove the else block if not needed
  return kor_filtered

In [85]:
# 실험
tfidf=TFIDF(example)
keyword_list = []
keyword_list=tfidf.run(vectorizer_type="tfidf",tokenizer=tokenizer)

['죽일려고', '하겠네', '쌩난리를', '한미반도체같이', '조단위인', '반도체를', '치더니', '영업익에', '만들고', '담주', '작년부터', '볼만', '쥐꼬리만한', '밀더니', '기형적인']
['방산', '방역당국', '방산업체향', '방산주', '방송에서', '방안을', '시외', '등락율', '순위', '시장이', '최종마감', '우선주의', '비정상적', 'ㅋㅋ', '날이군요']
['합작공장', '말했다', '세액공제', '공유', '최대', '생산', '북미', '알려졌다', '미국', '배당', '합작법인', '관계자는', '것으로', '완성차', '배터리']
['방법으로', '방문해도', '하세요', '오늘은', '주가가', '최소', '추매', '성투', '흘러', '영업', '올랐으니', '하려고요', '끝일듯요', '내리네요', '내려야죠']
['회원', '종목방에', '종목추천글은', '제가', '청약', '청약신청했다고', '안되어서요', '넣고', '못하는건지', '눌렀는데', '되시나요', '왔는데', '문자는', '조회가', '청약신청내역에서']
['방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방문해도', '방문', '힘이', '셀트리온']
['방향', '방해하거나', '방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방문한', '시가총액']
['미친듯이', '윈도드레싱', '저렴한', '사이에', '생김', '아무래도', '왕왕있음', '현물사이', '지켜보다가', '가격에', '익절완료', '괴리율이', '주식선물과', '동시호가에', '주식선물은']
['방한', '방역당국', '방안이', '방안을', '방송에서', '방산주', '방산업체향', '방산', '방사선으로', '방법이', '방법으로', '방향', '힘이',

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def extract_topic_keyword(word_list):
    text = ' '.join(word_list)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray()[0]
    max_tfidf_index = tfidf_scores.argmax()
    topic_keyword = feature_names[max_tfidf_index]
    return topic_keyword

def calculate_cosine_similarity_between_words(word1, word2):
    vectorizer = TfidfVectorizer()
    # TF-IDF 벡터라이저를 사용하여 각 단어의 TF-IDF 벡터를 생성
    vectors = vectorizer.fit_transform([word1, word2]).toarray()

    # 두 단어의 TF-IDF 벡터를 가져와 코사인 유사도 계산
    similarity = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1, -1))

    return similarity[0, 0]

In [88]:
result = {}

# 리스트의 첫 요소의 keywords에서 extract_topic_keyword 함수로 토픽을 추출해 result 딕셔너리의 새 키로 정의하고 cont_date를 키 value 리스트에 추가
topic = extract_topic_keyword(keyword_list[0]['keywords'])
result[topic] = [keyword_list[0]['cont_date']]
# vector1 = TfidfVectorizer().fit_transform([' '.join(keyword_list[0]['keywords'])]).toarray()[0]

# 리스트의 두번째 이상 요소부터는 해당 요소의 keywords에서 extract_topic_keyword 함수로 토픽을 추출해 기존 result 딕셔너리의 토픽들과 각각 코사인 유사도 비교
for i in range(1, len(keyword_list)):
    topic = extract_topic_keyword(keyword_list[i]['keywords'])
    # vector2 = TfidfVectorizer().fit_transform([' '.join(keyword_list[i]['keywords'])]).toarray()[0]

    # 유사도를 계산하여 임계값을 설정하고, 일정 임계값 이상일 경우에만 유사한 토픽으로 판단
    similarity_threshold = 0.4
    most_similar_topic = None
    highest_similarity = 0

    for existing_topic in list(result.keys()):
        similarity= calculate_cosine_similarity_between_words(existing_topic, topic)

        # 현재 토픽과의 유사도가 최고 유사도보다 높으면 갱신
        if similarity > highest_similarity:
            most_similar_topic = existing_topic
            highest_similarity = similarity

    # 최고 유사도가 임계값을 넘을 경우에만 합치기
    if highest_similarity > similarity_threshold:
        result[most_similar_topic].append(keyword_list[i]['cont_date'])
    else:
        result[topic] = [keyword_list[i]['cont_date']]

# 결과 출력
print(result)

{'기형적인': ['2023.12.02. 00:14'], 'ㅋㅋ': ['2023.12.01. 19:22'], '것으로': ['2023.12.01. 16:40'], '끝일듯요': ['2023.12.01. 13:25'], '넣고': ['2023.12.01. 10:41'], '방문': ['2023.12.01. 09:13', '2023.10.31. 11:46'], '방문한': ['2023.11.30. 18:25', '2023.11.23. 21:42', '2023.11.07. 09:07', '2023.11.06. 08:18', '2023.11.01. 05:14'], '가격에': ['2023.11.30. 16:02'], '방법으로': ['2023.11.30. 13:27', '2023.11.20. 14:21', '2023.11.10. 20:22', '2023.11.06. 18:46', '2023.10.31. 14:47', '2023.10.30. 10:10'], '방문해도': ['2023.11.30. 09:03', '2023.11.24. 17:44'], '개화': ['2023.11.29. 21:06'], '가지치기': ['2023.11.29. 18:13'], 'ㅎㅎ': ['2023.11.29. 14:06', '2023.11.15. 07:56', '2023.09.20. 08:54', '2023.09.17. 17:00'], '가네요': ['2023.11.29. 11:37', '2023.11.03. 16:35'], '가지': ['2023.11.29. 09:40'], '공매도': ['2023.11.28. 22:30'], '관리': ['2023.11.28. 17:05'], '가지고있음': ['2023.11.28. 14:35'], '걱정': ['2023.11.28. 09:51'], '고생하시는': ['2023.11.27. 19:05'], '가는게': ['2023.11.27. 15:23'], '경남제약': ['2023.11.27. 10:36', '2023.11.07. 18:47'], '

In [89]:
result

{'기형적인': ['2023.12.02. 00:14'],
 'ㅋㅋ': ['2023.12.01. 19:22'],
 '것으로': ['2023.12.01. 16:40'],
 '끝일듯요': ['2023.12.01. 13:25'],
 '넣고': ['2023.12.01. 10:41'],
 '방문': ['2023.12.01. 09:13', '2023.10.31. 11:46'],
 '방문한': ['2023.11.30. 18:25',
  '2023.11.23. 21:42',
  '2023.11.07. 09:07',
  '2023.11.06. 08:18',
  '2023.11.01. 05:14'],
 '가격에': ['2023.11.30. 16:02'],
 '방법으로': ['2023.11.30. 13:27',
  '2023.11.20. 14:21',
  '2023.11.10. 20:22',
  '2023.11.06. 18:46',
  '2023.10.31. 14:47',
  '2023.10.30. 10:10'],
 '방문해도': ['2023.11.30. 09:03', '2023.11.24. 17:44'],
 '개화': ['2023.11.29. 21:06'],
 '가지치기': ['2023.11.29. 18:13'],
 'ㅎㅎ': ['2023.11.29. 14:06',
  '2023.11.15. 07:56',
  '2023.09.20. 08:54',
  '2023.09.17. 17:00'],
 '가네요': ['2023.11.29. 11:37', '2023.11.03. 16:35'],
 '가지': ['2023.11.29. 09:40'],
 '공매도': ['2023.11.28. 22:30'],
 '관리': ['2023.11.28. 17:05'],
 '가지고있음': ['2023.11.28. 14:35'],
 '걱정': ['2023.11.28. 09:51'],
 '고생하시는': ['2023.11.27. 19:05'],
 '가는게': ['2023.11.27. 15:23'],
 '경남제약': 